#  Matching Catalogs by Proximity with NumCosmo

---
**License**

 Matching Catalogs by Proximity with NumCosmo

 Mon Dec 2 18:30:00 2024\
 Copyright  2024\
 Cinthia Nunes de Lima <cinthia.n.lima@uel.br> \ Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---
 matching_by_proximity\
 Copyright (C) 2024 Cinthia Nunes de Lima <cinthia.n.lima@uel.br>, Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

## Libraries & Configs

In [2]:
import math
import sys

import os
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table
import pandas as pd

sys.path.insert(0, "/global/homes/c/cinlima/gcrcatalogs-new/gcr-catalogs")
sys.path.insert(0, "/global/homes/c/cinlima/")
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/MachingCatalogs")


import GCRCatalogs
#Put nersc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")

print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)


from numcosmo_py import Nc, Ncm

import time

# GCRCatalogs.get_available_catalog_names()


GCRCatalogs = 1.9.0 | GCR = 0.9.2


## Catalogs

In [ ]:
#Cosmo DC2
cosmodc2= GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')

#Cosmo DC2 + RedMaPPer
catalog_dc2_redmapper = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')

In [4]:
# Restrictions on richness, mass and redshift
min_richness = 5
min_halo_mass = 1e13 #Msun
redshift_max = 1.2

#cDC2 + RM data
quantities = catalog_dc2_redmapper.list_all_quantities()
cluster_quantities = [q for q in quantities if 'member' not in q]
member_quantities = [q for q in quantities if 'member' in q]
    
query = GCRCatalogs.GCRQuery('(richness > ' + str(min_richness) +')')
cluster_data = Table(catalog_dc2_redmapper.get_quantities(cluster_quantities, [query])) # cDC2 + RM cluster data
member_data = Table(catalog_dc2_redmapper.get_quantities(member_quantities)) # cDC2 + RM member data

#cDC2 data
quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central']
query = GCRCatalogs.GCRQuery('(halo_mass > ' + str(min_halo_mass) +') & (redshift < ' + str(redshift_max) +') & (is_central < ' + str(redshift_max) +') ')
truth_data = Table(cosmodc2.get_quantities(quantities_wanted, [query]))

In [5]:
truth_data

dec,galaxy_id,halo_id,halo_mass,redshift,is_central,ra
float64,int64,int64,float64,float64,bool,float64
-40.400894272468605,9437500128,4500151475,16891355712901.41,0.04818676962281132,True,49.6366034309591
-40.41371737691802,9437500129,4500151475,16891355712901.41,0.04920109901409253,False,49.46521866823849
-40.40525333868696,9437500130,4500151475,16891355712901.41,0.04825475916728461,False,49.62952487674635
-40.40213407288883,9437500131,4500151475,16891355712901.41,0.04908434162264519,False,49.63722268629932
-40.40661788509503,9437500132,4500151475,16891355712901.41,0.04715899608329677,False,49.47215142449812
-40.389274415076976,9437500133,4500151475,16891355712901.41,0.04834164860201673,False,49.62774364073635
-40.383940630337435,9437500134,4500151475,16891355712901.41,0.04834778668489825,False,49.7276378027726
-40.27724451449116,9437500135,4500151475,16891355712901.41,0.04829776417102405,False,49.698541467041395
-40.54445769390596,9437500136,4500151475,16891355712901.41,0.04739776740602353,False,49.718255793095075


In [6]:
cluster_data

id_cen_1,richness_err,richness,p_cen_2,p_cen_0,redshift,id_cen_0,maskfrac,ra,redshift_err,ra_cen_0,dec_cen_0,id_cen_4,dec_cen_1,dec_cen_3,scaleval,ra_cen_3,cluster_id,dec,id_cen_3,dec_cen_4,p_cen_4,ra_cen_4,p_cen_1,ra_cen_2,id_cen_2,p_cen_3,dec_cen_2,redshift_true_cg,ra_cen_1
int64,float32,float32,float32,float32,float32,int64,float32,float64,float32,float64,float64,int64,float64,float64,float32,float64,int32,float64,int64,float64,float32,float64,float32,float64,int64,float32,float64,float32,float64
1312932492,3.7469666,190.70665,0.00010373697,0.79138356,0.5135129,1312945563,0.0,52.520436430812914,0.0027861558,52.520436430812914,-25.717996124671657,1313029113,-25.71783277825776,-25.72165959330821,1.0001404,52.527338368381535,10,-25.717996124671657,1312945763,-25.727465056757442,3.0111536e-07,52.51910078836549,0.20851088,52.50825021974284,1312932494,1.5151493e-06,-25.716281990601974,0.51072353,52.519490104517175
2000524824,3.6934361,179.12813,1.8312696e-06,0.99996513,0.5712697,2000524808,0.0,56.920376528125075,0.0032183963,56.920376528125075,-27.20434071042304,2000536915,-27.199286148618782,-27.20003879579277,1.0005391,56.92348931320997,18,-27.20434071042304,2000536980,-27.211258176404048,1.7345036e-08,56.92602537669649,3.297315e-05,56.94124732225573,2000525108,5.984015e-08,-27.204497548758997,0.5740297,56.927010885055374
2002535425,2.4172914,124.02715,0.026989846,0.93527913,1.040338,2002535198,0.0,56.99849698116052,0.0065176566,56.99849698116052,-26.7172081871169,2002535397,-26.718600369742887,-26.719138984756995,1.000476,56.98989225804766,24,-26.7172081871169,2002541935,-26.72317780680594,0.00042643907,56.999685738888104,0.03203878,57.00496745151956,2002535403,0.0052658287,-26.707287415263906,1.0347759,57.0029430173563
2000760286,3.6777163,167.33603,0.0005243642,0.9958548,0.63518524,2000759779,0.0,58.56362402841767,0.0060331975,58.56362402841767,-27.429961828331713,2000760130,-27.440237841615975,-27.44671248175573,1.0015866,58.56448660673249,30,-27.429961828331713,2000760140,-27.441554361218707,6.2297656e-10,58.54795919291436,0.0036076254,58.57655423797249,2000760010,1.3199483e-05,-27.4342933293566,0.6426174,58.55617219161547
2000292501,3.366554,111.9147,0.008745701,0.7297119,0.44230303,2000289335,0.0,58.35519782866568,0.003674398,58.35519782866568,-26.867687128773035,2000292608,-26.866270557202213,-26.864486047315793,1.0000473,58.35345231858996,71,-26.867687128773035,2000292504,-26.873768459561727,3.132921e-05,58.34775850906022,0.2614682,58.358819114610974,2000292571,4.2903914e-05,-26.87437529777802,0.43919736,58.34992458608299
2000290835,2.9570596,104.439766,6.477703e-05,0.99970347,0.46697217,2000290819,0.0,57.76575707007693,0.0035005552,57.76575707007693,-28.119101936484828,2000290977,-28.13711378351597,-28.112182213542138,1.0014223,57.77313021619973,73,-28.119101936484828,2000291009,-28.130550627706278,8.6784746e-07,57.78520845355392,0.00017803152,57.76312485832799,2000290930,5.2879543e-05,-28.123262961841775,0.4680865,57.75748991205223
2000286723,3.3524604,103.49396,3.0540557e-09,0.99999905,0.4413634,2000286447,0.0,58.509466974185045,0.0036729097,58.509466974185045,-26.989676182075844,2000286533,-27.01259056457905,-26.99741400320372,1.0021384,58.50636920080823,109,-26.989676182075844,2000286566,-27.003486551712935,1.5482234e-09,58.49500042467478,9.2802935e-07,58.4977149963539,2000286516,2.2677467e-09,-27.00358318558357,0.44349608,58.48648688545692
1940324773,2.2175314,76.85877,0.0004447735,0.6544826,1.0629631,1940324795,0.0,54.494027152041674,0.0070620407,54.494027152041674,-27.896589555863095,1940324995,-27.89888605332762,-27.890609698478475,1.0017645,54.47973899232569,116,-27.896589555863095,1940324997,-27.89828451222208,1.6846216e-05,54.498303753402574,0.34501392,54.48695011233644,1940324810,4.1882024e-05,-27.90097379323573,1.0758829,54.496631285075914
2002528410,2.26323,75.89771,0.00023012425,0.991116,1.037697,2002528355,0.0,58.137368555958155,0.0069939015,58.137368555958155,-27.199784884537003,2002528509,-27.2005263

## Matching by Proximity

In [ ]:
snn = Ncm.SphereNN()
    cosmo = Nc.HICosmoDEXcdm()
    dist = Nc.Distance.new(3.0)
    dist.prepare(cosmo)

    r_a = np.array([dist.comoving(cosmo, z) for z in z1])

    snn.insert_array(r_a, theta1, phi1)

    snn.rebuild()

    #
    # for theta, phi in zip(theta2, phi2):
    #    snn.knn_search(theta, phi, 10)
    # Do the above using tqdm

    for i, (theta, phi, z) in tqdm.tqdm(
        enumerate(zip(theta2, phi2, z2)), total=len(theta2)
    ):
        r = dist.comoving(cosmo, z)
        indices = np.array(snn.knn_search(r, theta, phi, 100))